# Lecture 2

## 背包问题

### 0/1 背包问题

给定数组 w, v。每个数组均有 N 个正整数。 w[i] 代表第 i 种东西的重量, v[i] 代表第 i 种东西的价值。

给定约束，总重量不能超过 C(整数), 东西不能重复拿的情况下, 求解如何背东西(一个数组 items), 以及其最大价值(一个数字).

我们将这个过程序列化,从左到右,每个 item 依次询问是否加入背包.这有点像是对于 episodic 的过程的时间戳的建模.
这样,状态空间就变为: (start_idx, capacity).

**version 1** 简单 DP

- 时间复杂度: O(N * C)
- 空间复杂度: O(N * C)

In [1]:
def backpack(w, v, C):
    N = len(w)
    dp = [[0] * (C+1) for _ in range(N)]
    # dp base
    for c in range(w[N-1], C+1):
        dp[N-1][c] = v[N-1]
    # dp base
    for i in range(N-2, -1, -1):
        for c in range(C+1):
            dp[i][c] = dp[i+1][c]
            if w[i] <= c:
                dp[i][c] = max(dp[i+1][c], v[i] + dp[i+1][c-w[i]])
    return dp[0][C]

w = [1, 3, 4, 5]
v = [15, 20, 30, 35]
C = 7

backpack(w, v, C)

50

由于这里的 dp 只是从最后一列传播到第一列,因此这里的空间复杂度可以进一步降低为: O(C)

**version 2** 滑动数组

- 时间复杂度: O(N*C)
- 空间复杂度: O(C)

In [2]:
def backpack(w, v, C):
    N = len(w)
    dp = [0] * (C+1)
    # dp base
    for c in range(w[N-1], C+1):
        dp[c] = v[N-1]
    # dp base
    for i in range(N-2, -1, -1):
        new_dp = dp.copy()
        for c in range(C+1):
            if w[i] <= c:
                new_dp[c] = max(dp[c], v[i] + dp[c-w[i]])
        dp = new_dp
    return dp[C]

w = [1, 3, 4, 5]
v = [15, 20, 30, 35]
C = 7

backpack(w, v, C)

50

背包问题还有其他一些问题:

1. 如果数据类型为 float 怎么办?

    我的思考是: 可以先 scale 到差距较大的情况下,再近似成整数.

2. 如果 C 非常大,这时创建一个这样的 dp 会很没效率,怎么办?

    我的思考是: 可以使用其他启发式算法,例如贪心, 或者使用列表作为字典的 key (当然, list 是 unhashable 的,但是可以考虑建立这样的辅助类来完成任务).

## 解码问题

状态空间:
  起始位置 i, 状态为 i,意味着要解码 code[i:]

In [3]:
def decode_count(code, alphabet):
    dp = [0] * (len(code) + 1)
    dp[len(code)] = 1
    i = len(code) - 1
    not_find_last_alphabet = True
    while not_find_last_alphabet:
        matches = match(code[i:], alphabet)
        if len(matches) == 0:
            dp[i] = 0
        else:
            dp[i] = 1
            not_find_last_alphabet = False
        i -= 1
    while i>=0:
        matches = match(code[i:], alphabet)
        for prefix in matches:
            dp[i] += dp[i+len(prefix)]
        i -= 1
    return dp[0]

def match(code, alphabet):
    prefix = []
    for a in alphabet:
        if code.startswith(a):
            prefix.append(a)
    return prefix

In [4]:
code = "111111"
alphabet = set([str(i+1) for i in range(26)])

decode_count(code, alphabet)

13

刚才写的是使用于一般编码的

下面再写一个专门为 digit 和英语字母对应解码的

In [5]:
def decode_count_digit(code, alphabet):
    # dp init
    dp = [0] * (len(code) + 1)
    dp[len(code)] = 1  # for naturally ending
    dp[len(code) - 1] = 1  # must match

    # dp sweep
    for i in range(len(code)-2, -1, -1):
        if code[i] == '0':
            continue
        dp[i] = dp[i+1]
        if int(code[i:i+2]) <= 26:
            dp[i] += dp[i+2]
    return dp[0]

In [6]:
code = "111111"
alphabet = set([str(i+1) for i in range(27)])
decode_count_digit(code, alphabet)

13

## 最长公共子序列问题

最长公共子序列问题可以使用动态规划算法

依然使用从左到右的状态空间建模, 假设字符串 a 的长度为 $n_1$, 字符串 b 的长度为 $n_2$.
那么我们的状态空间为 $S = \{1, .., n_1\} x \{1, ..., n_2\}$


**version 1** 暴力递归

In [24]:
def longest_common_seq(a, b):
    return longest_common_seq_recursive(a, b, len(a)-1, len(b)-1)

def longest_common_seq_recursive(a, b, end_a, end_b):
    # dp[0][0]
    if end_a == 0 and end_b == 0:
        return int(a[0] == b[0])
    # dp[0][*]
    elif end_a == 0:
        if a[end_a] == b[end_b]:
            return 1
        else:
            return longest_common_seq_recursive(a, b, end_a, end_b-1)
    # dp[*][0]
    elif end_b == 0:
        if a[end_a] == b[end_b]:
            return 1
        else:
            return longest_common_seq_recursive(a, b, end_a-1, end_b)

    # dp[i][j]
    elif a[end_a] == b[end_b]:
        return 1 + longest_common_seq_recursive(a, b, end_a-1, end_b-1)
    else:
        return max(longest_common_seq_recursive(a, b, end_a-1, end_b), longest_common_seq_recursive(a, b, end_a, end_b-1))

a = "shrugage"
b = "lbhguru"
longest_common_seq(a,b)

3

**version 2** 动态规划

In [40]:
def longest_common_seq(a, b):
    # we add a row and a col to store blank, dp init naturally done
    dp = [[0] * (len(b)+1) for _ in range(len(a)+1)]
    # dp sweep, row by row
    for i in range(1, len(a)+1):
        for j in range(1, len(b) + 1):
            if a[i-1] == b[j-1]:
                dp[i][j] = 1 + dp[i-1][j-1]
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    return dp[len(a)][len(b)]

a = "shrugage"
b = "lbhguru"
longest_common_seq(a,b)

3

## 象棋问题

![](./imgs/img.png)

In [75]:
W = 10 # x = 0, 1, ..., 9
H = 9  # y = 0, 1, ..., 8
def horse_jump(x, y, k):
    # dp init, calculate (*, *, 0)
    dp = [[[0]*(k+1) for _ in range(H)] for _ in range(W)]
    dp[x][y][0] = 1

    # dp sweep
    # calculate dp[i][j]
    for k_ in range(1, k+1):
        for i in range(W):
            for j in range(H):
                # calculate dp[i][j][k_]
                for (x_, y_) in next_state(i, j):
                    dp[i][j][k_] += dp[x_][y_][k_-1]
    # return
    return dp[0][0][k]
    # # you can return dp for debugging
    # return dp

def next_state(x, y):
    result = []
    s_1 = (x+1, y+2)
    s_2 = (x+1, y-2)
    s_3 = (x+2, y+1)
    s_4 = (x+2, y-1)
    s_5 = (x-1, y+2)
    s_6 = (x-1, y-2)
    s_7 = (x-2, y+1)
    s_8 = (x-2, y-1)
    new_states = [s_1, s_2, s_3, s_4, s_5, s_6, s_7, s_8]
    for s in new_states:
        if not out_board(s[0], s[1]):
            result.append(s)
    return result


def out_board(x, y):
    return x<0 or y<0 or x>W-1 or y>H-1

result = horse_jump(1, 2, 3)
result

8

Showing the $1^{st}$ layer of `dp`

In [72]:
import numpy as np
result = np.array(result)
result[:,:,1]

array([[1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

Showing the $2^{nd}$ layer of `dp`

In [73]:
result[:,:,2]

array([[0, 1, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 6, 0, 1, 0, 2, 0, 0],
       [0, 2, 0, 2, 0, 2, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 1, 0, 0],
       [0, 2, 0, 2, 0, 2, 0, 0, 0],
       [1, 0, 2, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

Showing the $3^{rd}$ layer of `dp`

In [74]:
result[:,:,3]

array([[ 8,  0,  6,  0, 12,  0,  3,  0,  2],
       [ 0,  5,  0,  8,  0,  4,  0,  4,  0],
       [11,  0,  8,  0, 17,  0,  4,  0,  3],
       [ 0, 14,  0, 18,  0,  8,  0,  6,  0],
       [ 6,  0,  6,  0,  9,  0,  3,  0,  1],
       [ 0,  4,  0,  6,  0,  3,  0,  3,  0],
       [ 4,  0,  6,  0,  6,  0,  3,  0,  0],
       [ 0,  3,  0,  3,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])

## 咖啡机问题

![](./imgs/img_1.png)
